<a href="https://colab.research.google.com/github/nagae/ICL_B_2021/blob/main/%E7%9C%8C%E5%88%A5%E3%83%BB%E5%B9%B4%E4%BB%A3%E5%88%A5%E3%81%AE%E6%AD%BB%E4%BA%A1%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib  import pyplot as plt, dates as mdates
import datetime

In [ ]:
orig_df = pd.read_json("https://raw.githubusercontent.com/reustle/covid19japan-data/master/docs/patient_data/latest.json")
#日付を文字列からdatetime型に変更しておく(列名を指定して代入)
orig_df["dateAnnounced"] = pd.to_datetime(orig_df["dateAnnounced"], format= "%Y-%m-%d")
orig_df["deceasedDate"] = pd.to_datetime(orig_df["deceasedDate"], format="%Y-%m-%d")
# ファイルを読み込み，列名(columns)を使いやすい形に修正

In [ ]:
df = orig_df.copy()
df.columns = [ "patientId", "年月日" , "年代", "性別", "都道府県", "容体", "感染順位", "都道府県別感染順位", "陽性数", "residence", "relatedPatients",	"knownCluster",	"detectedCityTown",	"cityPrefectureNumber",	"deceasedDate",	"deceasedReportedDate",	"deathSourceURL"]
# 年代データを整数値に変換する
for c in df.columns:
    if c not in ["年代"]: # 年代以外のデータは文字型として扱う
        df[c] = df[c].astype(str)
    else:
        df[c] = pd.to_numeric(df[c], errors='coerce') # 都道府県名以外のデータは数値として扱う
        df[c] = df[c].fillna(0) # データが欠損している部分を0で埋める
        df[c] = df[c].astype(int) # 一旦は実数型に変換する 
 
# 都道府県名と年月日からなるマルチインデックスを構築
df = df.set_index(["都道府県", "容体","年代", "年月日"])
df # orig_df の概観を表示

patientId  ... deathSourceURL
都道府県     容体           年代 年月日                   ...               
Kanagawa Recovered    30 2020-01-15        15  ...            nan
Tokyo    Recovered    40 2020-01-24      TOK1  ...            nan
                      30 2020-01-25      TOK2  ...            nan
Aichi    nan          40 2020-01-26        18  ...            nan
         Hospitalized 40 2020-01-28        19  ...            nan
...                                       ...  ...            ...
Osaka    Deceased     0  2021-06-29        -1  ...            nan
                         2021-06-29        -1  ...            nan
                         2021-06-29        -1  ...            nan
                         2021-06-29        -1  ...            nan
                         2021-06-29        -1  ...            nan

[812858 rows x 13 columns]

In [ ]:
dead_df = df.xs("Deceased",level="容体")["patientId"].groupby(level=["都道府県","年代","年月日"]).count().to_frame().unstack("年代")
columns = dead_df.columns.get_level_values(level=1).values.astype(str)
columns[columns == '0'] = "Unspecified"
dead_df.columns = columns
dead_df.fillna(0,inplace=True)
dead_df

Unspecified   20   30   40   50   60   70   80   90  100
都道府県      年月日                                                                 
Aichi     2020-02-29          0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
          2020-03-04          0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
          2020-03-05          0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
          2020-03-08          0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0
          2020-03-09          0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
...                           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
Yamanashi 2021-02-10          1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
          2021-02-27          1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
          2021-03-28          1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
          2021-06-07          1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
          2021-06-23          1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[4693 rows x 10 columns]

In [ ]:
dead_df.loc["Aichi"]

,Unspecified,20,30,40,50,60,70,80,90,100
年月日,,,,,,,,,,
2020-02-29,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2020-03-04,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2020-03-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2020-03-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2021-06-24,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-06-25,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-06-26,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 以下，未定

In [ ]:
dates = pd.to_datetime(df.index.get_level_values(level="年月日"))
start, end = (dates.min(), dates.max())
dates = np.arange(start,end,np.timedelta64(1,"D"))
prefs = df.index.get_level_values(level="都道府県").unique().drop(["Unspecified","Port Quarantine"])
ages = sorted(df.index.get_level_values(level="年代").unique())